In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itard"
fs_method, fs_ratio = "fcls", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9157 | AUPR: 0.6570 | Acc: 95.41%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9187 | AUPR: 0.6637 | Acc: 95.40%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9212 | AUPR: 0.6716 | Acc: 95.46%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9157 | AUPR: 0.6545 | Acc: 95.38%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9184 | AUPR: 0.6619 | Acc: 95.37%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9210 | AUPR: 0.6730 | Acc: 95.57%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9212 | AUPR: 0.6716 | Acc: 95.46%
Internal test: AUC: 0.9294 | AUPR: 0.7079 | Acc: 95.26%
External test: AUC: 0.9233 | AUPR: 0.6729 | Acc: 95.12%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9156 | AUPR: 0.6255 | Acc: 95.10%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9156 | AUPR: 0.6162 | Acc: 95.05%
model(kernel=linear, C=1)
Valid:         AUC: 0.9147 | AUPR: 0.6124 | Acc: 95.05%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9156 | AUPR: 0.6255 | Acc: 95.10%
Internal test: AUC: 0.9248 | AUPR: 0.6716 | Acc: 95.04%
External test: AUC: 0.9177 | AUPR: 0.6155 | Acc: 94.62%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7635 | AUPR: 0.4310 | Acc: 94.72%
model(n_neighbors=5)
Valid:         AUC: 0.7856 | AUPR: 0.4816 | Acc: 95.11%
model(n_neighbors=7)
Valid:         AUC: 0.7989 | AUPR: 0.5042 | Acc: 95.06%
best model(n_neighbors=7)
Valid:         AUC: 0.7989 | AUPR: 0.5042 | Acc: 95.06%
Internal test: AUC: 0.8049 | AUPR: 0.5209 | Acc: 94.91%
External test: AUC: 0.8101 | AUPR: 0.5102 | Acc: 94.91%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8737 | AUPR: 0.4083 | Acc: 86.46%
best model()
Valid:         AUC: 0.8737 | AUPR: 0.4083 | Acc: 86.46%
Internal test: AUC: 0.8834 | AUPR: 0.4362 | Acc: 86.28%
External test: AUC: 0.8704 | AUPR: 0.3873 | Acc: 84.86%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9165 | AUPR: 0.6508 | Acc: 95.33%
model(C=10.0)
Valid:         AUC: 0.9161 | AUPR: 0.6496 | Acc: 95.31%
model(C=100.0)
Valid:         AUC: 0.9157 | AUPR: 0.6492 | Acc: 95.32%
best model(C=1.0)
Valid:         AUC: 0.9165 | AUPR: 0.6508 | Acc: 95.33%
Internal test: AUC: 0.9252 | AUPR: 0.6972 | Acc: 95.33%
External test: AUC: 0.9158 | AUPR: 0.6407 | Acc: 94.88%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8732 | AUPR: 0.5588 | Acc: 84.16%
model(criterion=log_loss)
Valid:         AUC: 0.8623 | AUPR: 0.5623 | Acc: 82.94%
model(criterion=entropy)
Valid:         AUC: 0.8623 | AUPR: 0.5623 | Acc: 82.94%
best model(criterion=gini)
Valid:         AUC: 0.8732 | AUPR: 0.5588 | Acc: 84.16%
Internal test: AUC: 0.8698 | AUPR: 0.6080 | Acc: 83.65%
External test: AUC: 0.8727 | AUPR: 0.5708 | Acc: 82.92%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9143 | AUPR: 0.6530 | Acc: 89.55%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9156 | AUPR: 0.6567 | Acc: 90.05%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9161 | AUPR: 0.6580 | Acc: 90.08%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9126 | AUPR: 0.6553 | Acc: 89.71%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9147 | AUPR: 0.6578 | Acc: 90.08%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9158 | AUPR: 0.6595 | Acc: 90.34%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9126 | AUPR: 0.6553 | Acc: 89.71%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9147 | AUPR: 0.6578 | Acc: 90.08%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9158 | AUPR: 0.6595 | Acc: 90.34%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9161 | AUPR: 0.6580 | Acc: 9

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9191 | AUPR: 0.6578 | Acc: 95.15%
model(criterion=squared_error)
Valid:         AUC: 0.9191 | AUPR: 0.6578 | Acc: 95.16%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9191 | AUPR: 0.6578 | Acc: 95.15%
Internal test: AUC: 0.9274 | AUPR: 0.6959 | Acc: 95.18%
External test: AUC: 0.9208 | AUPR: 0.6587 | Acc: 94.95%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9173 | AUPR: 0.6505 | Acc: 95.30%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9173 | AUPR: 0.6503 | Acc: 95.28%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9174 | AUPR: 0.6509 | Acc: 95.33%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9174 | AUPR: 0.6509 | Acc: 95.33%
Internal test: AUC: 0.9187 | AUPR: 0.6766 | Acc: 95.18%
External test: AUC: 0.9132 | AUPR: 0.6289 | Acc: 94.84%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)